In [11]:
import pandas as pd
import pickle
import re

In [12]:
file_type = 'thread'
field_name = 'title'

In [13]:
pickle_file = open(f'{file_type}_{field_name}_token_lists.pickle', "rb" )
corpus = pickle.load( pickle_file )
pickle_file.close()

In [14]:
clean_corpus = list()
for document in corpus:
    clean_document = list()
    for word in document:
        if len(word) > 1:
            clean_document.append(re.sub(r"[\?|!|\.|,:|_]", '', word))
    clean_corpus.append(clean_document)
corpus = clean_corpus

In [17]:
from gensim.models import Phrases
bigrams_ = set()
trigrams_ = set()
bigram = Phrases(corpus, min_count=1, threshold=2)
trigram = Phrases(bigram[corpus], min_count=1, delimiter=b'_')
for sent in corpus:
    bigrams_ = bigrams_.union({b for b in bigram[sent] if b.count('_') == 1})
    trigrams_ = trigrams_.union({t for t in trigram[bigram[sent]] if t.count('_') == 2})

In [18]:
f = open('phrases.txt','w')
for phrase in bigrams_:
    f.write(f'{phrase}\n')
for phrase in trigrams_:
    f.write(f'{phrase}\n')
f.close()